# Zelf-gedefinieerde datatypes

:::{admonition} Begrippen

- type-synoniem (alternatieve naam)
- data-type definitie: opsomming van alternatieve waarden
- alternatieven: constructors
- alternatieven met attributen

:::

## Eenvoudige datatypes: alternatieve waarden

De datatypes die we tot nu toe gezien hebben, zowel de elementaire types zoals getallen, als de samengestelde types lijsten en tupels, zijn voorgedefinieerd. Maar, in Haskell kunnen we ook zelf datatypes definiëren.

De eenvoudigste vorm van een datatype-definitie geeft een *opsomming van de namen* van de verschillende waarden. Deze waarde-namen heten ook wel *constructors* - waarom, zal straks duidelijker worden.
Deze alternatieven kunnen ook van extra waarden voorzien zijn, zoals we verderop zullen zien.

**De namen van types en de namen van constructors beginnen in Haskell met een hoofdletter.**

### Voorbeeld: kleuren

In [45]:
data Color = Red | Orange | Yellow | Green | Blue | Indigo | Violet deriving (Show)

De namen van de alternatieven beginnen altijd met een hoofdletter. Zo'n naam heet een *constructor*, omdat je daarmee een waarde van het betreffende data-type maakt.

De toevoeging `deriving (Show)` betekent dat de `show`-functie voor deze waarden automatisch gedefinieerd wordt; deze laat de waarde zien zoals je deze in een Haskell-programma noteert. Deze `show`-functie wordt impliciet gebruikt om het resultaat van een cel te tonen.

We kunnen een kleur-waarde toekennen aan een naam, bijvoorbeeld:

In [46]:
myColor = Yellow

In [47]:
show myColor

"Yellow"


In [48]:
myColor

Yellow


We kunnen functies definiëren om met deze waarden te "rekenen". Eigenlijk moeten we alle vormen van rekenen met deze waarden nu zelf definiëren. De functies die we zo definiëren moeten voor elke Color-waarde een resultaat definiëren. We kunnen dat in Haskell doen door een definitie per Color-alternatief te geven:

In [50]:
nextColor :: Color -> Color
nextColor Red = Orange
nextColor Orange = Yellow
nextColor Yellow = Green
nextColor Green = Blue
nextColor Indigo = Violet
nextColor Violet = Violet

De structuur van deze functie volgt de structuur van het datatype van de parameter.

In [52]:
nextColor (nextColor myColor)

Blue


We hebben hier de functie `nextColor` gedefinieerd door een definitie voor elke mogelijke vorm van `Color`. Deze constructie heet ook wel *pattern matching*.

Een alternatief is om een "case analysis" binnen de functie-definitie uit te voeren, zoals in dit voorbeeld:

```Haskell
nextColor :: Color -> Color
nextColor c = 
    case c of
    Red -> Orange
    Orange -> Yellow 
    Yellow -> Green
    Green -> Blue
    Indigo -> Violet
    Violet -> Violet
```


### Bool als data-type

Een aantal types zijn voorgedefinieerd in de Haskell "standard prelude": de standaard-library van Haskell. Het type Bool is daarin bijvoorbeeld gedefinieerd als:

```Haskell
data  Bool  =  False | True
```

## Alternatieven met attributen

### Voorbeeld: vormen

In het eenvoudige data-type `Color` spreken de waarden voor zich. Maar bij complexere types kunnen de alternatieven *attributen* hebben.

Een voorbeeld hiervan is het type `Shape` (voor een 2-dimensionale geometrische vorm). We onderscheiden (in eerste instantie) cirkels en rechthoeken:

* een cirkel heeft een middelpunt (punt) en de straal (Float)
* een rechthoek een positie (punt, voor de linker-bovenhoek), een breedte en een hoogte (Floats)
* een punt is een *2-tupel* van Floats: x- en y-coördinaten.

Als afkorting voeren we het type `Point` in: een tupel 2 getallen: de x- en y-coördinaat van het punt.

In [1]:
type Point = (Float, Float)

> Dit is een voorbeeld van een **type-synoniem**: overal waar de naam `Point` gebruikt wordt, kun je ook `(Float, Float)` schrijven of lezen.

We onderscheiden twee vormen: een cirkel, met middelpunt en straal; en een rechthoek, met positie, breedte en hoogte.

In [2]:
data Shape = Circle Point Float | Rect Point Float Float

**Uitzoeken** *(Hebben we geen andere manier om deze eigenschappen te benoemen en te documenteren?)*

**Uitzoeken**: attributen zoals lijndikte, lijnkleur en vulkeur zijn eigenlijk gemeenschappelijk voor alle vormen. Hoe kun je dat het best uitdrukken in Haskell? Een vorm van "overerving"?

Later zullen we toevoegen als vormen:

* lijnstuk (met 2 coördinaten: begin- en eindpunt)
* pad (een lijst van coördinaten; reeks aaneengesloten lijnstukken)
* tekst

We kunnen nu een functie definiëren voor het uitrekenen van de oppervlakte van een (gesloten) figuur. We moeten in die functie de verschillende soorten vormen onderscheiden. In Haskell kan dat door de functie voor elk alternatief afzonderlijk te definiëren:

In [3]:
area :: Shape -> Float
area (Circle centre radius) = pi * radius * radius
area (Rect position width height) = width * height

We demonstreren dit met twee voorbeeld-vormen:

In [19]:
shape1 = Circle (50, 50) 20
shape2 = Rect (20, 30) 100 20

In [11]:
area shape1

314.15927


In [12]:
area shape2

200.0


We kunnen nog meer functies definiëren voor deze vormen

- `translate :: Point -> Shape -> Shape`
- `scale :: Float -> Shape -> Shape`
- `tosvg :: Shape -> String`

Als voorbeeld werken we deze laatste functie uit: `tosvg`, om de SVG-string voor deze vorm te bepalen. Deze kunnen we dan kopiëren in een SVG-figuur, om deze te tonen. (Zie het svg-display notebook.)

De attributen van een SVG-element hebben allemaal dezelfde structuur: `name="value"`. Hier worden dezelfde dubbele quote-tekens gebruikt als voor Haskell-strings Dat betekent dat we deze niet zomaar in een string kunnen opnemen: we hebben een *escape*-notatie nodig. De volgende functie maakt deze structuur aan: 


In [30]:
attr :: String -> Float -> String
attr name value = name ++ "=\"" ++ (show value) ++ "\" "

de functie `show 2.3` zet het getal om in een string, hier `"2.3"`

In [36]:
attr "cx" 12.34

"cx=\"12.34\" "


In [41]:
tosvg :: Shape -> String
tosvg (Circle (mx, my) r) = "<circle " ++ (attr "cx" mx) ++ (attr "cy" my) ++ (attr "r" r) ++ " /> \n"
tosvg (Rect (mx, my) w h) = "<rect " ++ (attr "x" mx) ++ (attr "y" my) ++ (attr "width" w) ++ (attr "height" h) ++ " />"

In [42]:
tosvg shape1

"<circle cx=\"50.0\" cy=\"50.0\" r=\"20.0\"  /> \n"


Om dit te kunnen gebruiken in een SVG-figuur, moeten we deze string-waarde in de uitvoer-vorm hebben, in plaats van in de Haskell-notatie; met andere woorden, zonder de quote-tekens, en met de escape-tekens zooals `'\n'` en `'\"'` verwerkt. Hiervoor gebruiken we de functie `putStr`:

In [43]:
putStr (tosvg shape1)

<circle cx="50.0" cy="50.0" r="20.0"  /> 


In [44]:
putStr (tosvg shape2)

<rect x="20.0" y="10.0" width="10.0" height="20.0"  />

De inhoud van deze cel kopiëren we naar de lege regel in de cel met `svgimage=...`, in het svg-display notebook. (Zie de handleiding daar.)
Voor een figuur met `shape1` en `shape2` geeft dit:

![](svg-fig1.svg)

**Opdrachten**

- toevoegen van andere (SVG) vormen, bijvoorbeeld *ellipse* en *line*.
    - je moet dan per vorm het alternatief toevoegen aan het type; en de bijbehorende functies uitbreiden.
- toevoegen van extra attributen aan de SVG-vormen, bijvoorbeeld de vul-kleur

### Voorbeeld: expressies

Als tweede voorbeeld van zelf-gedefineerde datatypes behandelen we *expressies*. We willen expressies als data kunnen opbouwen. We maken hier een eerste begin, en zullen zien dat deze vorm nogal beperkt is. Voor 